In [17]:
"""
Collaborative Filtering ALS Recommender System using Spark MLlib adapted from
the Spark Summit 2014 Recommender System training example.

Developed By: Pranav Masariya
Supervisor: Dr. Magdalini Eirinaki
"""

import os
import numpy as np
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pyspark.mllib.recommendation import ALS
from pyspark.ml.recommendation import ALS as mlals
from pyspark.ml.evaluation import RegressionEvaluator

import math
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator

In [18]:
# Calling spark session to register application
spark = SparkSession \
    .builder \
    .appName("Recom") \
    .config("spark.recom.demo", "1") \
    .getOrCreate()

In [19]:
"""
Loading and Parsing Dataset
    Each line in the ratings dataset (ratings.csv) is formatted as:
         userId,movieId,rating,timestamp
    Each line in the movies (movies.csv) dataset is formatted as:
        movieId,title,genres

""" 

# Load ratings
ratings_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("ml-latest-small/ratings.csv")

In [20]:
"""
For the simplicity of this tutorial
    For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). 
    We drop the timestamp because we do not need it for this recommender.
"""

ratings_df = ratings_df.drop('timestamp')
ratings_df.show(30)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     31|   2.5|
|     1|   1029|   3.0|
|     1|   1061|   3.0|
|     1|   1129|   2.0|
|     1|   1172|   4.0|
|     1|   1263|   2.0|
|     1|   1287|   2.0|
|     1|   1293|   2.0|
|     1|   1339|   3.5|
|     1|   1343|   2.0|
|     1|   1371|   2.5|
|     1|   1405|   1.0|
|     1|   1953|   4.0|
|     1|   2105|   4.0|
|     1|   2150|   3.0|
|     1|   2193|   2.0|
|     1|   2294|   2.0|
|     1|   2455|   2.5|
|     1|   2968|   1.0|
|     1|   3671|   3.0|
|     2|     10|   4.0|
|     2|     17|   5.0|
|     2|     39|   5.0|
|     2|     47|   4.0|
|     2|     50|   4.0|
|     2|     52|   3.0|
|     2|     62|   3.0|
|     2|    110|   4.0|
|     2|    144|   3.0|
|     2|    150|   5.0|
+------+-------+------+
only showing top 30 rows



In [21]:
# Load movies
movies_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("ml-latest-small/movies.csv")

In [22]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [23]:
"""
For each line in the movies dataset, we create a tuple of (MovieID, Title). 
    We drop the genres because we do not use them for this recommender.
"""
movies_df = movies_df.drop('genres')
movies_df.show(5)

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|      1|    Toy Story (1995)|
|      2|      Jumanji (1995)|
|      3|Grumpier Old Men ...|
|      4|Waiting to Exhale...|
|      5|Father of the Bri...|
+-------+--------------------+
only showing top 5 rows



In [24]:
"""
In order to determine the best ALS parameters, we will use the small dataset. 
We need first to split it into train, validation, and test datasets.
"""
(trainingData,validationData,testData) = ratings_df.randomSplit([0.6,0.2,0.2])

In [25]:
# Prepare test and validation set. They should not have ratings

validation_for_predict = validationData.select('userId','movieId')
test_for_predict = testData.select('userId','movieId')

In [26]:
"""
Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by 
using Alternating Least Squares. The implementation in MLlib has the following parameters:

    1. numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
    2. rank is the number of latent factors in the model.
    3. iterations is the number of iterations to run.
    4. lambda specifies the regularization parameter in ALS.
    5. implicitPrefs specifies whether to use the explicit 
        feedback ALS variant or one adapted for implicit feedback data.
    6. alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline 
        confidence in preference observations.

"""

seed = 5L #Random seed for initial matrix factorization model. A value of None will use system time as the seed.
iterations = 10
regularization_parameter = 0.1 #run for different lambdas - e.g. 0.01
ranks = [4, 8, 12] #number of features
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

In [27]:
# Let us traing our dataset and check the best rank with lowest RMSE
# predictAll method of the ALS takes only RDD format and hence we need to convert our dataframe into RDD
# df.rdd will automatically converts Dataframe into RDD

for rank in ranks:
    model = ALS.train(trainingData, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validationData.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) # RMSE Error
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 0.94357979683
For rank 8 the RMSE is 0.953406650964
For rank 12 the RMSE is 0.949731159093
The best model was trained with rank 4


In [28]:
"""
Spark will soon deprecate MLLIb package. 
They are focusing more on ML packages with standard machine learning implementation
Let's see that package also
"""
als =  mlals(maxIter=iterations,rank=4,seed=seed,regParam=regularization_parameter, userCol="userId", itemCol="movieId",ratingCol="rating")
modelML = als.fit(trainingData)
pred = modelML.transform(validationData)
pred = pred.where(pred['prediction'] != 'NaN')
    
# Evaluate the model by computing RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(pred)

print 'RMSE is %s' % rmse

"""
The best part is we do not have to worry about RDD any more with this library
"""

RMSE is 0.9382325858


'\nThe best part is we do not have to worry about RDD any more with this library\n'

In [29]:
# Let's take test dataset and get ratings
predictions_test = model.predictAll(test_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))

In [30]:
## visualize preditions, here third element is predictions generated by ALS Model
predictions_test.take(3)

[((564, 1084), 4.301811687554226),
 ((580, 1084), 3.555661129941959),
 ((281, 1084), 3.981041517972427)]

In [31]:
"""
Let's start recommending movies.
I have written a method to call recommendations for a perticular user from test data

TODO: You need to execute one more step before calling getRecommendations, 
      Think about that step. If you go through the seps below, you will realize it soon.
"""
def getRecommendations(user,testDf,trainDf,model):
    # get all user and his/her rated movies
    userDf = testDf.filter(testDf.userId == user)
    # filter movies from main set which have not been rated by selected user
    # and pass it to model we sreated above
    mov = trainDf.select('movieId').subtract(userDf.select('movieId'))
    
    # Again we need to covert our dataframe into RDD
    pred_rat = model.predictAll(mov.rdd.map(lambda x: (user, x[0]))).collect()
    
    # Get the top recommendations
    recommendations = sorted(pred_rat, key=lambda x: x[2], reverse=True)[:50]
    
    return recommendations

In [32]:
# Assign user id for which we need recommendations
user = 336

# Call getRecommendations method
derived_rec = getRecommendations(user,testData,trainingData,model)

print "Movies recommended for:%d" % user

# Print the result
# TODO: we can convert derived_rec into a dataframe to present it properly
for i in xrange(len(derived_rec)):
    print i+1
    movies_df.filter(movies_df.movieId==derived_rec[i][1]).select('title').show()

Movies recommended for:336
1
+--------------------+
|               title|
+--------------------+
|Hachiko: A Dog's ...|
+--------------------+

2
+--------------+
|         title|
+--------------+
|Amistad (1997)|
+--------------+

3
+--------------------+
|               title|
+--------------------+
|Children of a Les...|
+--------------------+

4
+----------------+
|           title|
+----------------+
|Body Heat (1981)|
+----------------+

5
+--------------------+
|               title|
+--------------------+
|Maya Lin: A Stron...|
+--------------------+

6
+----------------+
|           title|
+----------------+
|Stalag 17 (1953)|
+----------------+

7
+--------------------+
|               title|
+--------------------+
|Murphy's Romance ...|
+--------------------+

8
+--------------------+
|               title|
+--------------------+
|Man Who Shot Libe...|
+--------------------+

9
+--------------------+
|               title|
+--------------------+
|Paradise Lost: Th...|
+----